# Section 1 :


In [26]:
#all the neseccary imports
import pandas as pd
from google.colab import files
import io

##  GROUPING ALL COMPANIES BASED ON MARKET CAPITALISATION 

In [27]:
# reading csv file
uploaded = files.upload()

Saving ind_nifty500list.csv to ind_nifty500list.csv


In [33]:
# reading csv file
uploaded = files.upload()

Saving MCAP.csv to MCAP.csv


In [28]:
df2 = pd.read_csv(io.BytesIO(uploaded['ind_nifty500list.csv']))
df2.head()
nifty500list=list(df2['Symbol'])

In [34]:
df = pd.read_csv(io.BytesIO(uploaded['MCAP.csv']))
df.head()

,Sr No,Symbol,Company Name,"Market capitalization as on December 31, 2021\n(Rs in Lakhs)"
0,1,RELIANCE,Reliance Industries Limited,160138207
1,2,TCS,Tata Consultancy Services Limited,138283487
2,3,HDFCBANK,HDFC Bank Limited,81983196
3,4,INFY,Infosys Limited,80445056
4,5,HINDUNILVR,Hindustan Unilever Limited,55453878


In [35]:
df= df.drop(['Sr No','Company Name'],axis = 1)
df.rename(columns = {'Market capitalization as on December 31, 2021\n(Rs in Lakhs)':'MCAP'}, inplace = True)

In [36]:
cap=[]
for i in df['MCAP']:
  try:
    price=(float(i)*0.01)
    if price>=20000:
      cap.append(0)
    elif price<20000 and price>5000:
      cap.append(1)
    elif price<5000:
      cap.append(2)
  except:  
    cap.append('NAN')
#print(cap)
cap.append('NAN')
cap.append('NAN')

In [37]:
df['CAP']=cap
df.drop(df[df['CAP'] == 'NAN'].index, inplace = True)

In [38]:
large_df = df[df['CAP'] == 0]
mid_df = df[df['CAP'] == 1]
small_df = df[df['CAP'] == 2]
large_list=list(large_df['Symbol'])
mid_list=list(mid_df['Symbol'])
small_list=list(small_df['Symbol'])

### **large_list** have all large cap companies
### **mid_list** have all mid cap companies
### **small_list** have all small cap companies

# Section 2 :


In [3]:
pip install yfinance

     |████████████████████████████████| 6.4 MB 7.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
#all the neseccary imports
import datetime as dt
import yfinance as yf
import pandas_datareader.data as pdr
import plotly.graph_objects as go
import math
import matplotlib.pyplot as plt
yf.pdr_override()

##  MAKING CHART

In [52]:
def get_sma(prices,rate):
  return prices.rolling(rate).mean()

In [53]:
def Boll_band(prices,rate=20):
  sma=get_sma(prices,rate)
  std=prices.rolling(rate).std()
  boll_up = sma + std*2
  boll_dw = sma - std*2  
  return boll_up,boll_dw

In [54]:
def risk_ana(entry_price,stop_loss):
  risk=150
  sl=entry_price-stop_loss
  no_of_share=math.ceil(risk/sl)
  target1=entry_price+sl
  target2=entry_price+(sl*2)

  return (risk,entry_price,stop_loss,no_of_share,target1,target2)

In [55]:
start = dt.datetime.now()- dt.timedelta(days=100)
end = dt.datetime.now()    

In [56]:
#moving Average
def MA(st,start,end,Int,rate):
  if Int=='15m':
    return False
  else:
    d = pdr.get_data_yahoo(st,start,end,interval='1d')
    d = d.reset_index()
    d = d.drop(['Volume'],axis = 1)
    ma50 = get_sma(d.Close,rate)
    d['MA']=ma50
    #d=d.dropna(axis=0)
    #print(ma50)
    Ma=list(d['MA'])
    #print("an",ma50)
    #print("an",list(d.Close))
    #rising
    Ma1=list(d['MA'])[-1000:]
    date_list=list(d.Date)
    open_list=list(d.Open)
    high_list=list(d.High)
    low_list=list(d.Low)
    close_list=list(d.Close)
    bol_up,bol_dw=Boll_band(d.Close)
  return (date_list,open_list,high_list,low_list,close_list,Ma,Ma1,bol_up,bol_dw,d['Close'])


# **STRATEGY ABC**

In [57]:
def make_fig(strat,st):
  fig=go.Figure()
  if strat=="ABC":
    Date=[]
    Open=[]
    High=[]
    Close=[]
    Low=[]
    rising=[]
    Org_Ma = []
    st="{0}.NS".format(i)
    Date,Open,High,Low,Close,Org_Ma,rising,bwl_up,bwl_dw,df=MA(st,start,end,'1d',50)
    if len(Org_Ma)==0:
      print("enot getting moving average ",i)
      print("(hint : inc time frame)")
      #break
    else:
      last_bwl_dw=list(bwl_dw)[-1]
      last_close=list(Close)[-1]
      last_open=list(Open)[-1]
      last_high=list(High)[-1]
      last_low=list(Low)[-1]
      last_low2=list(Low)[-2]
      dt=str(end.date())
      check=rising[-1]-last_bwl_dw
      if rising[-1]>rising[-3]:
        if last_bwl_dw>rising[-1] or check<=12:
          #if (((0.6< last_close-last_bwl_dw) and  ((last_close-last_bwl_dw)<100)) and ((0.6< float(last_close-rising[-1])) and (float(last_close-rising[-1])<100))):
          if (((0.0001 < ((last_close-last_bwl_dw)/last_close)) and  ( ((last_close-last_bwl_dw)/last_close)<0.03)) and ((0.0001< float((last_close-rising[-1])/last_close)) and (float((last_close-rising[-1])/last_close)<0.03))):
            #GBC
            if last_close>last_open:
              per=(((last_high-last_low)/last_high)*100)
              if ((0<int(round(per,1)))or(int(round(per,1))<=6)):  
                fig.update_layout(title_text=i +" CLOSE: "+str(round(last_close,3))+" OPEN: "+
                        str(round(list(Open)[-1],3))+" HIGH: "+str(round(list(High)[-1],3))+
                        " LOW: "+str(round(list(Low)[-1],3))+
                        " AS ON "+str(end.date())+" STRATEGY - ABC (GBC)")
                fig.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
                fig.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
                fig.update_layout(spikedistance=800, hoverdistance=100,height=800)
                fig.add_trace(
                      go.Candlestick(
                      name="Market Price",
                      x=Date,
                      open=Open, high=High,
                      low=Low, close=Close))
                fig.add_trace(
                      go.Scatter(
                      name="50MA",
                      x=Date,
                      y=Org_Ma,
                      line_width=3, opacity=1,
                      line_dash="dot", hoverinfo='skip'))
                fig.add_trace(
                      go.Scatter(
                      name="Boll_up",
                      x=Date,
                      y=bwl_up,hoverinfo='skip'))
                fig.add_trace(
                      go.Scatter(
                      name="Boll_down",
                      x=Date,
                      y=bwl_dw,hoverinfo='skip'))
                fig.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
                fig.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
                fig.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
                fig.show()
                if last_low>last_low2:
                  fin=last_low2
                else:
                  fin=last_low
                risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)          
                print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
                flag=True
                ##return flag
            #hammer  
            elif ((((last_close==last_high)or(last_close>=last_high-2))and (last_open>=(2*last_low)))
                    or  (((last_open==last_high)or(last_open>=last_high-2)) and (last_close>=(2*last_low)))):
              fig.update_layout(title_text=i +" CLOSE: "+str(round(last_close,3))+" OPEN: "+
                        str(round(list(Open)[-1],3))+" HIGH: "+str(round(list(High)[-1],3))+
                        " LOW: "+str(round(list(Low)[-1],3))+
                        " AS ON "+str(end.date())+" STRATEGY - ABC (hammer)")
              fig.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
              fig.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
              fig.update_layout(spikedistance=800, hoverdistance=100,height=800)
 
              fig.add_trace(
                      go.Candlestick(
                      name="Market Price",
                      x=Date,
                      open=Open, high=High,
                      low=Low, close=Close))
              fig.add_trace(
                      go.Scatter(
                      name="50MA",
                      x=Date,
                      y=Org_Ma,
                      line_width=3, opacity=1,
                      line_dash="dot", hoverinfo='skip'))
              fig.add_trace(
                      go.Scatter(
                      name="Boll_up",
                      x=Date,
                      y=bwl_up,hoverinfo='skip'))
              fig.add_trace(
                      go.Scatter(
                      name="Boll_down",
                      x=Date,
                      y=bwl_dw,hoverinfo='skip'))
              fig.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
              fig.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
              fig.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
              fig.show()
              if last_low>last_low2:
                fin=last_low2
              else:
                fin=last_low
              risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
              print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
              flag=True
              #return flag

        

  #elif strat=="MA":
  
  #elif strat=="BB":

  

In [60]:
j=0
for i in nifty500list:
  make_fig('ABC',i)
  
  j+=1
  print(i)
  print(j)
  #break

[*********************100%***********************]  1 of 1 completed
DEVYANI
1
[*********************100%***********************]  1 of 1 completed
INDIAMART
2
[*********************100%***********************]  1 of 1 completed
SUNTV
3
[*********************100%***********************]  1 of 1 completed
AJANTPHARM
4
[*********************100%***********************]  1 of 1 completed
SUMICHEM
5
[*********************100%***********************]  1 of 1 completed
L&TFH
6
[*********************100%***********************]  1 of 1 completed
SUNDRMFAST
7
[*********************100%***********************]  1 of 1 completed
HAPPSTMNDS
8
[*********************100%***********************]  1 of 1 completed
PRESTIGE
9
[*********************100%***********************]  1 of 1 completed
BCG
10
[*********************100%***********************]  1 of 1 completed
CARBORUNIV
11
[*********************100%***********************]  1 of 1 completed
NATIONALUM
12
[*********************100%************

# **STRATEGY 44MA**  

In [66]:
top=[]
j,k=0,0
for i in mid_list:
  Date=[]
  Open=[]
  High=[]
  Close=[]
  Low=[]
  rising=[]
  Org_Ma = []
  st="{0}.NS".format(i)
  
  Date,Open,High,Low,Close,Org_Ma,rising,bwl_up,bwl_dw,df=MA(st,start,end,'1d',44)
  if len(Org_Ma)==0:
    print("error",i)
    break
  else:
    last_bwl_dw=list(bwl_dw)[-1]
    last_close=list(Close)[-1]
    last_open=list(Open)[-1]
    last_high=list(High)[-1]
    last_low=list(Low)[-1]
    last_low2=list(Low)[-2]
    if rising[-1]>rising[-3]:
      if (((0.6< last_close-last_bwl_dw) and  ((last_close-last_bwl_dw)<100)) and 
        ((0.6< float(last_close-rising[-1])) and (float(last_close-rising[-1])<100))):
        #GBC
        if last_close>last_open:
          per=(((last_close-last_open)/last_open)*100)
          if last_close-last_open>0 and ((0<int(round(per,1)))or(int(round(per,1))<=6)):  
            fig1 = go.Figure()
            fig1.update_layout(title_text=i +" CLOSE: "+str(round(last_close,3))+" OPEN: "+
                        str(round(list(Open)[-1],3))+" HIGH: "+str(round(list(High)[-1],3))+
                        " LOW: "+str(round(list(Low)[-1],3))+
                        " AS ON "+str(end.date())
                        +" STRATEGY - 44ma")
            fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
            fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
            fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)
            fig1.add_trace(
              go.Candlestick(
              name="Market Price",
              x=Date,
              open=Open, high=High,
              low=Low, close=Close))
            fig1.add_trace(
            go.Scatter(
              name="44MA",
            x=Date,
            y=Org_Ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
            fig1.update_traces(line_color="darkmagenta", selector=dict(name="44MA",type='scatter'))
            fig1.show()
            if last_low>last_low2:
              fin=last_low2
            else:
              fin=last_low
            risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
            print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
            top.append(i)
            #break
          #Hammer
        elif ((((0<=last_high-last_close)and(1>=last_high-last_close))and((last_close-last_open)*2 <=(last_open-last_low))) or 
        (((0<=last_high-last_open)and(1>=last_high-last_open))and((last_open-last_close)*2 <=(last_close-last_low)))):
          
          fig1 = go.Figure()
          fig1.update_layout(title_text=i +" CLOSE: "+str(round(last_close,3))+" OPEN: "+
                        str(round(list(Open)[-1],3))+" HIGH: "+str(round(list(High)[-1],3))+
                        " LOW: "+str(round(list(Low)[-1],3))+
                        " AS ON "+str(end.date())+" STRATEGY - 44MA")
          fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
          fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
          fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)
 
          fig1.add_trace(
            go.Candlestick(
              name="Market Price",
              x=Date,
              open=Open, high=High,
              low=Low, close=Close))
          fig1.add_trace(
            go.Scatter(
              name="44MA",
            x=Date,
            y=Org_Ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
          fig1.update_traces(line_color="darkmagenta", selector=dict(name="44MA",type='scatter'))
          fig1.show()
          top.append(i)
          if last_low>last_low2:
            fin=last_low2
          else:
            fin=last_low
          risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)          
          print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
            #break
        else:
            print("NO PATTERN",i)
      else:
          print("GAP",i)
    else:
      print("No Rising",i)
      

  j+=1
  print(j)
  #break


[*********************100%***********************]  1 of 1 completed
No Rising DEVYANI
1
[*********************100%***********************]  1 of 1 completed
No Rising INDIAMART
2
[*********************100%***********************]  1 of 1 completed
No Rising SUNTV
3
[*********************100%***********************]  1 of 1 completed
No Rising AJANTPHARM
4
[*********************100%***********************]  1 of 1 completed
NO PATTERN SUMICHEM
5
[*********************100%***********************]  1 of 1 completed
No Rising L&TFH
6
[*********************100%***********************]  1 of 1 completed
No Rising SUNDRMFAST
7
[*********************100%***********************]  1 of 1 completed
No Rising HAPPSTMNDS
8
[*********************100%***********************]  1 of 1 completed
No Rising PRESTIGE
9
[*********************100%***********************]  1 of 1 completed
No Rising BCG
10
[*********************100%***********************]  1 of 1 completed
No Rising CARBORUNIV
11
[*********

Risk-- 150
ENTRY PRICE-- 1878.050048828125
Stop Loss-- 1836.1500244140625
NO OF SHARES-- 4
TARGET 1:01 -- 1919.9500732421875
TARGET 1:02 -- 1961.85009765625
19
[*********************100%***********************]  1 of 1 completed
No Rising METROPOLIS
20
[*********************100%***********************]  1 of 1 completed
NO PATTERN FEDERALBNK
21
[*********************100%***********************]  1 of 1 completed
No Rising INDIANB
22
[*********************100%***********************]  1 of 1 completed
No Rising GILLETTE
23
[*********************100%***********************]  1 of 1 completed
No Rising PHOENIXLTD
24
[*********************100%***********************]  1 of 1 completed
NO PATTERN APTUS
25
[*********************100%***********************]  1 of 1 completed
No Rising GRINFRA
26
[*********************100%***********************]  1 of 1 completed
GAP KPITTECH
27
[*********************100%***********************]  1 of 1 completed
NO PATTERN POONAWALLA
28
[********************

Risk-- 150
ENTRY PRICE-- 927.5999755859375
Stop Loss-- 911.2999877929688
NO OF SHARES-- 10
TARGET 1:01 -- 943.8999633789062
TARGET 1:02 -- 960.199951171875
31
[*********************100%***********************]  1 of 1 completed
No Rising CHAMBLFERT
32
[*********************100%***********************]  1 of 1 completed
No Rising WABCOINDIA
33
[*********************100%***********************]  1 of 1 completed
No Rising APLLTD
34
[*********************100%***********************]  1 of 1 completed
No Rising SFL
35
[*********************100%***********************]  1 of 1 completed
GAP KIOCL
36
[*********************100%***********************]  1 of 1 completed
No Rising CDSL
37
[*********************100%***********************]  1 of 1 completed
No Rising UCOBANK
38
[*********************100%***********************]  1 of 1 completed
No Rising ABSLAMC
39
[*********************100%***********************]  1 of 1 completed
No Rising BSOFT
40
[*********************100%*****************

Risk-- 150
ENTRY PRICE-- 663.0
Stop Loss-- 637.0
NO OF SHARES-- 6
TARGET 1:01 -- 689.0
TARGET 1:02 -- 715.0
58
[*********************100%***********************]  1 of 1 completed
No Rising CAMS
59
[*********************100%***********************]  1 of 1 completed
No Rising MAHABANK
60
[*********************100%***********************]  1 of 1 completed
No Rising FINPIPE
61
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 31.350000381469727
Stop Loss-- 26.299999237060547
NO OF SHARES-- 30
TARGET 1:01 -- 36.400001525878906
TARGET 1:02 -- 41.450002670288086
62
[*********************100%***********************]  1 of 1 completed
No Rising FSL
63
[*********************100%***********************]  1 of 1 completed
No Rising QUESS
64
[*********************100%***********************]  1 of 1 completed
GAP IIFLWAM
65
[*********************100%***********************]  1 of 1 completed
NO PATTERN SUVENPHAR
66
[*********************100%***********************]  1 of 1 completed
No Rising METROBRAND
67
[*********************100%***********************]  1 of 1 completed
No Rising MEDPLUS
68
[*********************100%***********************]  1 of 1 completed
NO PATTERN KEC
69
[*********************100%***********************]  1 of 1 completed
No Rising CHOLAHLDNG
70
[*********************100%***********************]  1 of 1 completed
No Rising TATAMTRDVR
71
[*********************100%***

...

Risk-- 150
ENTRY PRICE-- 494.8999938964844
Stop Loss-- 413.6000061035156
NO OF SHARES-- 2
TARGET 1:01 -- 576.1999816894531
TARGET 1:02 -- 657.4999694824219
222
[*********************100%***********************]  1 of 1 completed
GAP POLYPLEX
223
[*********************100%***********************]  1 of 1 completed
No Rising TATVA
224
[*********************100%***********************]  1 of 1 completed
No Rising THYROCARE
225
[*********************100%***********************]  1 of 1 completed
NO PATTERN EASEMYTRIP
226
[*********************100%***********************]  1 of 1 completed
No Rising GODFRYPHLP
227
[*********************100%***********************]  1 of 1 completed
No Rising GOCOLORS
228
[*********************100%***********************]  1 of 1 completed
No Rising TCI
229
[*********************100%***********************]  1 of 1 completed
No Rising KRBL
230
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 481.0
Stop Loss-- 466.45001220703125
NO OF SHARES-- 11
TARGET 1:01 -- 495.54998779296875
TARGET 1:02 -- 510.0999755859375
231
[*********************100%***********************]  1 of 1 completed
No Rising MAZDOCK
232
[*********************100%***********************]  1 of 1 completed
No Rising BARBEQUE
233
[*********************100%***********************]  1 of 1 completed
NO PATTERN KALPATPOWR
234
[*********************100%***********************]  1 of 1 completed
No Rising BURGERKING
235
[*********************100%***********************]  1 of 1 completed
No Rising JCHAC
236
[*********************100%***********************]  1 of 1 completed
NO PATTERN INFIBEAM
237
[*********************100%***********************]  1 of 1 completed
NO PATTERN TRIVENI
238
[*********************100%***********************]  1 of 1 completed
No Rising RALLIS
239
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 1449.449951171875
Stop Loss-- 1319.0
NO OF SHARES-- 2
TARGET 1:01 -- 1579.89990234375
TARGET 1:02 -- 1710.349853515625
240
[*********************100%***********************]  1 of 1 completed
GAP ESABINDIA
241
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 509.79998779296875
Stop Loss-- 482.29998779296875
NO OF SHARES-- 6
TARGET 1:01 -- 537.2999877929688
TARGET 1:02 -- 564.7999877929688
242
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 309.79998779296875
Stop Loss-- 297.20001220703125
NO OF SHARES-- 12
TARGET 1:01 -- 322.39996337890625
TARGET 1:02 -- 334.99993896484375
243
[*********************100%***********************]  1 of 1 completed
No Rising HEIDELBERG
244
[*********************100%***********************]  1 of 1 completed
NO PATTERN VARROC
245
[*********************100%***********************]  1 of 1 completed
No Rising JYOTHYLAB
246
[*********************100%***********************]  1 of 1 completed
No Rising AARTIDRUGS
247


# **STRATEGY BOLLINGER BAND** **BUY**

In [63]:
top=[]
j,k=0,0
for i in mid_list:
  Date=[]
  Open=[]
  High=[]
  Close=[]
  Low=[]
  rising=[]
  Org_Ma = []
  st="{0}.NS".format(i)
  Date,Open,High,Low,Close,Org_Ma,rising,bwl_up,bwl_dw,adjclose=MA(st,start,end,'1d',50)
  
  if len(Org_Ma)==0:
    print("error",i)
    break
  else:
    last_bwl_dw=list(bwl_dw)[-1]
    last2_bwl_dw=list(bwl_dw)[-2]
    last_close=list(Close)[-1]
    last2_close=list(Close)[-2]
    last_open=list(Open)[-1]
    last2_open=list(Open)[-2]
    last_high=list(High)[-1]
    last_low=list(Low)[-1]
    last2_low=list(Low)[-2]
    #RED
    #print(last2_open,last2_close)
    #print(last2_bwl_dw,last2_close)
    if last2_open>last2_close and last2_bwl_dw>last2_close:  
      if last_close>last_open and last_close>last_bwl_dw:
        fig2 = go.Figure()
        fig2.update_layout(title_text=i +" OPEN: "+str(round(last_open,3))+" HIGH: "+
                        str(round(list(High)[-1],3))+" LOSE: "+str(round(list(Low)[-1],3))+
                        " CLOSE: "+str(round((last_close),3))+
                        " AS ON "+str(end.date())
                        +" STRATEGY - BB")
        fig2.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
        fig2.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
        fig2.update_layout(spikedistance=800, hoverdistance=100,height=800)
        fig2.add_trace(
          go.Candlestick(
              name="Market Price",
              x=Date,
              open=Open, high=High,
              low=Low, close=Close))
        fig2.add_trace(
            go.Scatter(
              name="Boll_up",
            x=Date,
            y=bwl_up,hoverinfo='skip'
            ))
        fig2.add_trace(
            go.Scatter(
              name="Boll_down",
            x=Date,
            y=bwl_dw,hoverinfo='skip'
            ))
        fig2.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
        fig2.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
        fig2.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
        fig2.show()
        if last_low>last2_low:
          fin=last2_low
        else:
          fin=last_low
        risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)
        print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
        top.append(i)
      else:
        print("no Green",i)
    else:
      print("no red candel",i)  
      
          
      
  j+=1
  print(j)
  #break

[*********************100%***********************]  1 of 1 completed
180.0 179.89999389648438
164.84598975691836 179.89999389648438
no red candel DEVYANI
1
[*********************100%***********************]  1 of 1 completed
5130.0 5139.0
4105.384366724303 5139.0
no red candel INDIAMART
2
[*********************100%***********************]  1 of 1 completed
521.0 521.5999755859375
474.90221034014286 521.5999755859375
no red candel SUNTV
3
[*********************100%***********************]  1 of 1 completed
2076.0 2058.85009765625
2031.2351170245008 2058.85009765625
no red candel AJANTPHARM
4
[*********************100%***********************]  1 of 1 completed
412.79998779296875 410.70001220703125
366.16334053906934 410.70001220703125
no red candel SUMICHEM
5
[*********************100%***********************]  1 of 1 completed
75.8499984741211 75.75
71.61268920393047 75.75
no red candel L&TFH
6
[*********************100%***********************]  1 of 1 completed
848.0 847.3499755859375
8

Risk-- 150
ENTRY PRICE-- 796.0
Stop Loss-- 752.2999877929688
NO OF SHARES-- 4
TARGET 1:01 -- 839.7000122070312
TARGET 1:02 -- 883.4000244140625
123
[*********************100%***********************]  1 of 1 completed
582.2999877929688 576.9000244140625
504.58411863279576 576.9000244140625
no red candel CGCL
124
[*********************100%***********************]  1 of 1 completed
11770.0 11177.849609375
9912.555301307862 11177.849609375
no red candel LAXMIMACH
125
[*********************100%***********************]  1 of 1 completed
588.0 568.0499877929688
529.3214733313737 568.0499877929688
no red candel JUBLINGREA
126
[*********************100%***********************]  1 of 1 completed
2357.25 2364.75
2129.8317209848415 2364.75
no red candel ECLERX
127
[*********************100%***********************]  1 of 1 completed
2190.449951171875 2235.39990234375
1760.0106088381415 2235.39990234375
no red candel RATNAMANI
128
[*********************100%***********************]  1 of 1 completed


Risk-- 150
ENTRY PRICE-- 1449.449951171875
Stop Loss-- 1319.0
NO OF SHARES-- 2
TARGET 1:01 -- 1579.89990234375
TARGET 1:02 -- 1710.349853515625
240
[*********************100%***********************]  1 of 1 completed
3096.5 3110.60009765625
3011.3813050051303 3110.60009765625
no red candel ESABINDIA
241
[*********************100%***********************]  1 of 1 completed
504.95001220703125 497.1499938964844
438.69695526966325 497.1499938964844
no red candel GREENPANEL
242
[*********************100%***********************]  1 of 1 completed
303.79998779296875 300.1499938964844
275.5767645701384 300.1499938964844
no red candel FDC
243
[*********************100%***********************]  1 of 1 completed
215.14999389648438 214.8000030517578
213.42589191931367 214.8000030517578
no red candel HEIDELBERG
244
[*********************100%***********************]  1 of 1 completed
408.5 415.3999938964844
382.66316489808844 415.3999938964844
no red candel VARROC
245
[*********************100%******

In [ ]:
print(len(top))

8


# **STRATEGY BOLLINGER BAND** **SELL**

In [ ]:
top=[]
j,k=0,0
for i in large_list:
  Date=[]
  Open=[]
  High=[]
  Close=[]
  Low=[]
  rising=[]
  Org_Ma = []
  st="{0}.NS".format(i)
  Date,Open,High,Low,Close,Org_Ma,rising,bwl_up,bwl_dw=MA(st,start,end,'1d',50)
  if len(Org_Ma)==0:
    print("error",i)
    break
  else:
    last_bwl_dw=list(bwl_dw)[-1]
    last2_bwl_dw=list(bwl_dw)[-2]
    last_bwl_up=list(bwl_up)[-1]
    last2_bwl_up=list(bwl_up)[-2]
    last_close=list(Close)[-1]
    last2_close=list(Close)[-2]
    last_open=list(Open)[-1]
    last2_open=list(Open)[-2]
    last_high=list(High)[-1]
    last_low=list(Low)[-1]
    #Green
    if last2_close>last2_open and last2_close>last2_bwl_up:  
      if last_open>last_close and last_bwl_up>last_close:
        fig2 = go.Figure()
        fig2.update_layout(title_text=i +" CLOSE: "+str(round(last_close,3))+" OPEN: "+
                        str(round(list(Open)[-1],3))+" HIGH: "+str(round(list(High)[-1],3))+
                        " LOW: "+str(round(list(Low)[-1],3))+
                        " AS ON "+str(end.date())
                        +" STRATEGY - BB SELL")
        fig2.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
        fig2.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
        fig2.update_layout(spikedistance=800, hoverdistance=100,height=800)
        fig2.add_trace(
          go.Candlestick(
              name="Market Price",
              x=Date,
              open=Open, high=High,
              low=Low, close=Close))
        fig2.add_trace(
            go.Scatter(
              name="Boll_up",
            x=Date,
            y=bwl_up,hoverinfo='skip'
            ))
        fig2.add_trace(
            go.Scatter(
              name="Boll_down",
            x=Date,
            y=bwl_dw,hoverinfo='skip'
            ))
        fig2.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
        fig2.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
        fig2.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
        fig2.show()
        risk,ep,sl,nos,tg1,tg2=risk_ana(last_close,last_open-10)          
        print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
        top.append(i)
        break
      else:
        print("no red",i)
    else:
      print("no green",i)  
      
          
      
  j+=1
  print(j)
  #break

NameError: ignored

# **next**

In [ ]:
timelist=["09:15"," 09:30"," 09:45"," 10:00"," 10:15"," 10:30"," 10:45"," 11:00"," 11:15"," 11:30"," 11:45"," 12:00"," 12:15"," 12:30",
" 12:45"," 13:00"," 13:15"," 13:30"," 13:45"," 14:00"," 14:15"," 14:30"," 14:45"," 15:00"," 15:15"]
print(len(timelist))

25


In [ ]:
start1 = dt.datetime.now()- dt.timedelta(days=10)
end1 = dt.datetime.now()
j=0
top=[]
for i in nifty500list:
  st="{0}.NS".format(i)
  h = pdr.get_data_yahoo(st,start1,end1,interval='15m')
  h = h.reset_index()
  #h['time']=timelist
  h = h.drop(['Volume'],axis = 1)
  ma50 = get_sma(h.Close,50)
  h['MA']=ma50
  #d=d.dropna(axis=0)
  #print(ma50)
  Ma=list(h['MA'])
  Ma1=list(h['MA'])[-100:]
  date_list1=list(h.Datetime)
  last_date_list=date_list1[-30:]
  last_ma=Ma1[-30:]
  open_list1=list(h.Open)
  high_list1=list(h.High)
  low_list1=list(h.Low)
  close_list1=list(h.Close)
  bol_up1,bol_dw1=Boll_band(h.Close[-50:])
  last_bwl_dw=list(bol_dw1)[-1]
  last2_bwl_dw=list(bol_dw1)[-2]
  last_bwl_up=list(bol_up1)[-1]
  last2_bwl_up=list(bol_up1)[-2]
  last_close=list(close_list1)[-1]
  last2_close=list(close_list1)[-2]
  last_open=list(open_list1)[-1]
  last2_open=list(open_list1)[-2]
  last_high=list(high_list1)[-1]
  last_low=list(low_list1)[-1]
  last_low2=list(low_list1)[-2]
    #Green
  print(j)  
  if Ma[-1]>Ma[-25]:
    if (((0.6< last_close-last_bwl_dw) and  ((last_close-last_bwl_dw)<100)) and 
        ((0.6< float(last_close-Ma1[-1])) and (float(last_close-Ma1[-1])<100))):
        #GBC
        if last_close>last_open:
          per=(((last_close-last_open)/last_open)*100)
          if last_close-last_open>0 and ((0<int(round(per,1)))or(int(round(per,1))<=6)):  
            fig1 = go.Figure()
            fig1.update_layout(title_text=i )
            fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
            fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
            fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)
          
            fig1.add_trace(
              go.Candlestick(
              name="Market Price",
              x=timelist,
              open=open_list1[-25:], high=high_list1[-25:],
              low=low_list1[-25:], close=close_list1[-25:]))
            fig1.add_trace(
              go.Scatter(
              name="50MA",
            x=timelist,
            y=last_ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
            fig1.add_trace(
              go.Scatter(
              name="Boll_up",
              x=timelist,
              y=bol_up1[-25:],hoverinfo='skip'
              ))
            fig1.add_trace(
              go.Scatter(
              name="Boll_down",
              x=timelist,
              y=bol_dw1[-25:],hoverinfo='skip'
              ))
            fig1.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
            fig1.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
            fig1.update_traces(line_color="darkmagenta", selector=dict(name="44MA",type='scatter'))
            fig1.show()
            top.append(i)
            if last_low>last_low2:
              fin=last_low2
            else:
              fin=last_low
            risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
            print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
            #break
          #Hammer
        elif ((((last_close==last_high)or(last_close>=last_high-2))and (last_open>=(2*last_low)))
                    or  (((last_open==last_high)or(last_open>=last_high-2)) and (last_close>=(2*last_low)))):
            
          fig1 = go.Figure()
          fig1.update_layout(title_text=i + " STRATEGY - 50ma")
          fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
          fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
          fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)

          fig1.add_trace(
            go.Candlestick(
              name="Market Price",
              x=timelist,
              open=open_list1[-25:], high=high_list1[-25:],
              low=low_list1[-25:], close=close_list1[-25:]))
          fig1.add_trace(
            go.Scatter(
              name="50MA",
            x=timelist,
            y=last_ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
          fig1.add_trace(
            go.Scatter(
              name="Boll_up",
            x=timelist,
            y=bol_up1[-25:],hoverinfo='skip'
            ))
          fig1.add_trace(
            go.Scatter(
              name="Boll_down",
            x=timelist,
            y=bol_dw1[-25:],hoverinfo='skip'
            ))
          
          
          fig1.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
          fig1.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
        
          fig1.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
          fig1.show()
          top.append(i)
          risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
          print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
          #break
        else:
            print("NO PATTERN",i)
      
      
      
      

  j+=1
  print(j)
  #break


# **50MA 15min**


In [ ]:
start1 = dt.datetime.now()- dt.timedelta(days=10)
start2 = dt.datetime.now()+dt.timedelta(days=2)
end1 = dt.datetime.now()
j=0
top=[]
for i in nifty500list:
  st="{0}.NS".format(i)
  h = pdr.get_data_yahoo(st,start1,end1,interval='15m')
  h1 = pdr.get_data_yahoo(st,"2022-02-11",start2,interval='15m')
  h = h.reset_index()
  h1 = h1.reset_index()
  #h['time']=timelist
  h = h.drop(['Volume'],axis = 1)
  h1 = h1.drop(['Volume'],axis = 1)
  ma50 = get_sma(h.Close,50)
  h['MA']=ma50
  #d=d.dropna(axis=0)
  #print(ma50)

  tp=int(h1.shape[0])
  
  Ma=list(h['MA'])
  Ma1=list(h['MA'])[-100:]
  date_list1=list(h.Datetime)
  
  last_date_list=date_list1[-tp:]
  last_ma=Ma1[-tp:]
  
  open_list1=list(h1.Open)
  high_list1=list(h1.High)
  low_list1=list(h1.Low)
  close_list1=list(h1.Close)
  bol_up1,bol_dw1=Boll_band(h.Close)
  bwl_up_list=list(bol_up1)
  bwl_dw_list=list(bol_dw1)
  
  last_bwl_dw=bwl_dw_list[-1]
  last2_bwl_dw=bwl_dw_list[-2]
  last_bwl_up=bwl_up_list[-1]
  last2_bwl_up=bwl_up_list[-2]
  last_close=close_list1[-1]
  last2_close=close_list1[-2]
  last_open=open_list1[-1]
  last2_open=open_list1[-2]
  last_high=high_list1[-1]
  last2_high=high_list1[-2]
  last_low=low_list1[-1]
  last_low2=low_list1[-2]

  today_bwl_up=bwl_up_list[-tp:]
  today_bwl_dw=bwl_dw_list[-tp:]

  #print(open_list1,high_list1,close_list1,low_list1)
    #Green
  #print(j)  
  #break
  if Ma1[-1]>Ma1[0]:
    if (((0.6< last_close-last_bwl_dw) and  ((last_close-last_bwl_dw)<100)) and 
        ((0.6< float(last_close-Ma1[-1])) and (float(last_close-Ma1[-1])<100))):
        #GBC
        if last2_close>last2_open:
          per=(((last_close-last_open)/last_open)*100)
          if last_close-last_open>0 and ((0<int(round(per,1)))or(int(round(per,1))<=6)):  
            fig1 = go.Figure()
            fig1.update_layout(title_text=i + " STRATEGY - 50ma GBC" )
            fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
            fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
            fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)
          
            fig1.add_trace(
              go.Candlestick(
              name="Market Price",
              x=timelist,
              open=open_list1, high=high_list1,
              low=low_list1, close=close_list1))
            fig1.add_trace(
            go.Scatter(
              name="50MA",
            x=timelist[:tp],
            y=last_ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
            fig1.add_trace(
            go.Scatter(
              name="Boll_up",
            x=timelist[:tp],
            y=today_bwl_up,hoverinfo='skip'
            ))
            fig1.add_trace(
            go.Scatter(
              name="Boll_down",
            x=timelist[:tp],
            y=today_bwl_dw,hoverinfo='skip'
            ))
            fig1.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
            fig1.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
            fig1.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
            fig1.show()
            top.append(i)
            if last_low>last_low2:
              fin=last_low2
            else:
              fin=last_low
            #risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
            #print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
            break
          #Hammer
        elif ((((0<=last2_high-last2_close)and(1>=last2_high-last2_close))and((last2_close-last2_open)*2 <=(last2_open-last_low2))) or 
        (((0<=last2_high-last2_open)and(1>=last2_high-last2_open))and((last2_open-last2_close)*2 <=(last2_close-last_low2)))):
          fig1 = go.Figure()
          fig1.update_layout(title_text=i + " STRATEGY - 50ma Hammer")
          fig1.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
          fig1.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
          fig1.update_layout(spikedistance=800, hoverdistance=100,height=800)

          fig1.add_trace(
            go.Candlestick(
              name="Market Price",
              x=timelist,
              open=open_list1, high=high_list1,
              low=low_list1, close=close_list1))
          fig1.add_trace(
            go.Scatter(
              name="50MA",
            x=timelist[:tp],
            y=last_ma,
            line_width=3, opacity=1,
            line_dash="dot", hoverinfo='skip')
            )
          fig1.add_trace(
            go.Scatter(
              name="Boll_up",
            x=timelist[:tp],
            y=today_bwl_up,hoverinfo='skip'
            ))
          fig1.add_trace(
            go.Scatter(
              name="Boll_down",
            x=timelist[:tp],
            y=today_bwl_dw,hoverinfo='skip'
            ))
          fig1.update_traces(line_color="red", selector=dict(name="Boll_down",type='scatter'))
          fig1.update_traces(line_color="forestgreen", selector=dict(name="Boll_up",type='scatter'))
          
          fig1.update_traces(line_color="darkmagenta", selector=dict(name="50MA",type='scatter'))
          fig1.show()
          top.append(i)
          #risk,ep,sl,nos,tg1,tg2=risk_ana(last_high+1,fin-1)         
          #print("Risk-- "+str(risk)+"\n"+"ENTRY PRICE-- "+str(ep)+"\n"+"Stop Loss-- "+str(sl)+"\n"+"NO OF SHARES-- "+str(nos)+"\n"+"TARGET 1:01 -- "+str(tg1)+"\n"+"TARGET 1:02 -- "+str(tg2))
          #break
        else:
            print("NO PATTERN",i)
      
      
      

  j+=1
  print(j ,i)
  #break


In [ ]:
print(len(top))

1


## helper

In [ ]:
start2 = dt.datetime.now()+ dt.timedelta(days=1)
end2=dt.datetime.now()
h1 = pdr.get_data_yahoo("SBIN.NS","2022-02-11",start2,interval='15m')
#h1.info()
#h1.tail()
print(end2)
print(start2)
print(h1)
print(int(h1.shape[0]))
tp=int(h1.shape[0])
open_list1=list(h1.Open)
close_list1=list(h1.Close)
low_list1=list(h1.Low)
high_list1=list(h1.High)
last_ma=Ma1[-30:]
fig4 = go.Figure()
fig4.update_layout(title_text="SBIN.NS")
fig4.update_xaxes(showspikes=True,spikemode="across",spikethickness=1,showgrid = False)
fig4.update_yaxes(showspikes=True,spikethickness=1,spikedash="dash",showgrid = False)
fig4.update_layout(spikedistance=800, hoverdistance=100,height=800)
fig4.add_trace(
            go.Candlestick(
              name="Market Price",
              x=timelist,
              open=open_list1, high=high_list1,
              low=low_list1, close=close_list1))
          
fig4.update_traces(line_color="darkmagenta", selector=dict(name="44MA",type='scatter'))
fig4.update_traces(line_color="darkmagenta", selector=dict(name="44MA",type='scatter'))
fig4.show()
print(last_ma)
          

# **ATH**

In [51]:
j=0
for i in small_list:
  s="{0}.NS".format(i)
  y = pdr.get_data_yahoo(s,period="max",interval='1mo')
  j+=1
  try:
    print(j,i,max(list(y['High'])))
    y = y.reset_index()
    hgh=max(list(y['High']))
    per_close = (hgh-list(y['Close'])[-1])/(list(y['Close'])[-1]) 
    if (hgh == list(y['Close'])[-1]) or (0<hgh and hgh<0.15): 
      fig5 = go.Figure()
      fig5.update_layout(title_text=s)
      fig5.add_trace(
            go.Candlestick(
              name="Market Price",
              x=list(y['Date']),
              open=list(y['Open']), high=list(y['High']),
              low=list(y['Low']), close=list(y['Close'])))
      fig5.add_hline(y=hgh)
      fig5.show()
      break
  except:
      pass

[*********************100%***********************]  1 of 1 completed
1 ICIL 315.0
[*********************100%***********************]  1 of 1 completed
2 CEATLTD 2019.0
[*********************100%***********************]  1 of 1 completed
3 VSTIND 4848.0
[*********************100%***********************]  1 of 1 completed
4 MAHLOG 821.0
[*********************100%***********************]  1 of 1 completed
5 GSFC 166.39999389648438
[*********************100%***********************]  1 of 1 completed
6 BHARATRAS 15131.7001953125
[*********************100%***********************]  1 of 1 completed
7 GPPL 261.8999938964844
[*********************100%***********************]  1 of 1 completed
8 BOROLTD 498.70001220703125
[*********************100%***********************]  1 of 1 completed
9 TCNSBRANDS 932.0
[*********************100%***********************]  1 of 1 completed
10 CRAFTSMAN 2778.0
[*********************100%***********************]  1 of 1 completed
11 TEJASNET 570.5
[*************

In [48]:
y = pdr.get_data_yahoo('EMAMIPAP.NS',period="max",interval='1mo')
y 

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-02-11,152.5,155.0,150.699997,153.25,153.25,24592


#TRY

In [7]:
import yfinance as yf

msft = yf.Ticker("SBIN.NS")

# get stock info
msft.info

# get historical market data
#hist = msft.history(period="max")

#print(hist)

{'52WeekChange': 0.30138838,
 'SandP52WeekChange': 0.12359536,
 'address1': 'State Bank Bhavan',
 'address2': '14th Floor Corporate Centre Madame Cama Road, Nariman Point',
 'algorithm': None,
 'annualHoldingsTurnover': None,
 'annualReportExpenseRatio': None,
 'ask': 0,
 'askSize': 0,
 'averageDailyVolume10Day': 25761173,
 'averageVolume': 18390221,
 'averageVolume10days': 25761173,
 'beta': 0.974025,
 'beta3Year': None,
 'bid': 0,
 'bidSize': 0,
 'bookValue': 338.239,
 'category': None,
 'circulatingSupply': None,
 'city': 'Mumbai',
 'companyOfficers': [],
 'country': 'India',
 'currency': 'INR',
 'currentPrice': 529.6,
 'currentRatio': None,
 'dateShortInterest': None,
 'dayHigh': 537.7,
 'dayLow': 527.1,
 'debtToEquity': None,
 'dividendRate': 4,
 'dividendYield': 0.0075,
 'earningsGrowth': 0.528,
 'earningsQuarterlyGrowth': 0.527,
 'ebitda': None,
 'ebitdaMargins': 0,
 'enterpriseToEbitda': None,
 'enterpriseToRevenue': 3.913,
 'enterpriseValue': 8237560102912,
 'exDividendDate': 

In [8]:
pip install nsepy

  Created wheel for nsepy: filename=nsepy-0.8-py3-none-any.whl size=36084 sha256=61ceb1187a1d64a99b64b98a21024855cdd2d9ff7f164190573c9d2846d72111
  Stored in directory: /root/.cache/pip/wheels/32/ab/d9/78ceea14cdf6de83376082b3cb0c2999fd77f823e35c47b9ec
Successfully built nsepy


In [58]:
y = pdr.get_data_yahoo(".NS",period="max",interval='1mo')
print(max(list(y['High'])))

[*********************100%***********************]  1 of 1 completed
1279.260009765625
